# A swift improvement of Taylor's emissions

## Imports

In [74]:
import numpy as np
import pandas as pd

In [75]:
!pip install ortools

In [76]:
from ortools.linear_solver import pywraplp
import time

## Functions

### Function that calculates the ecological impact per mode of transport

In [77]:
def ecological_impact(distance, transport_type):
    emissions_per_km = {
        'Bus (average)': 96.5,
        'Coach (bus)': 27.33,
        'Diesel car': 170.82,
        'Electric car': 47.09,
        'Motorbike': 113.55,
        'National rail': 35.49,
        'Petrol car': 170.48,
        'Plug-in hybrid': 68.4,
        'Short-haul flight': 245.87,
        'Medium-haul flight': 151.02,
        'Long-haul flight': 147.87,
        'Private Jet (Short-haul) flight': 2458.70,
        'Private Jet (Medium-haul) flight': 1510.20,
        'Private Jet (Long-haul) flight': 1478.70
    }
    if 'flight' in transport_type and 'Private Jet' not in transport_type:
        if distance < 1500:
            transport_type = 'Short-haul flight'
        elif distance < 4000:
            transport_type = 'Medium-haul flight'
        else:
            transport_type = 'Long-haul flight'
    elif 'Private Jet' in transport_type:
        if distance < 1500:
            transport_type = 'Private Jet (Short-haul) flight'
        elif distance < 4000:
            transport_type = 'Private Jet (Medium-haul) flight'
        else:
            transport_type = 'Private Jet (Long-haul) flight'
    return emissions_per_km.get(transport_type, 0) * distance

###Function that gives the average speed per mode of transport:

In [78]:
def speed(transport_type,distance):
    speed = {
        'Bus (average)': 60,
        'Coach (bus)': 65,
        'Diesel car': 80,
        'Electric car': 80,
        #'Ferry (foot passenger)': 30,
        'Motorbike': 70,
        'National rail': 60,
        'Petrol car': 80,
        'Plug-in hybrid': 80,
        'Tram': 10,
        'Short-haul flight': 200,
        'Medium-haul flight': 250,
        'Long-haul flight': 300,
        'Private Jet (Short-haul) flight': 250,
        'Private Jet (Medium-haul) flight': 300,
        'Private Jet (Long-haul) flight': 450
    }

    if 'flight' in transport_type and 'Private Jet' not in transport_type:
        if distance < 1500:
            transport_type = 'Short-haul flight'
        elif distance < 4000:
            transport_type = 'Medium-haul flight'
        else:
            transport_type = 'Long-haul flight'
    elif 'Private Jet' in transport_type:
        if distance < 1500:
            transport_type = 'Private Jet (Short-haul) flight'
        elif distance < 4000:
            transport_type = 'Private Jet (Medium-haul) flight'
        else:
            transport_type = 'Private Jet (Long-haul) flight'

    return speed.get(transport_type, float('inf'))

###Function that gives the distance between two cities

Flying distances

In [79]:
import math

def haversine(coord1, coord2):
    # Radius of the Earth in kilometers
    R = 6371.0
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def create_distance_matrix_flight(location_names, locations):
    n = len(location_names)
    distance_matrix = [[0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i != j:
                loc1 = tuple(map(float, locations[i].split(',')))
                loc2 = tuple(map(float, locations[j].split(',')))
                distance_matrix[i][j] = haversine(loc1, loc2)

    return distance_matrix

Routing distances with Open Source Routing Machine (OSRM)

In [80]:
import requests

def get_route(start_coord, end_coord):
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coord};{end_coord}?overview=false"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Handle non-200 HTTP responses
        routes = response.json()['routes'][0]
        distance = routes['distance'] / 1000  # Convert to kilometers
        return distance
    except requests.RequestException as e:
        print(f"Error querying OSRM: {e}")
        return 0  # Return a default value or handle otherwise

def create_distance_matrix_routing(location_names, locations):
    distance_matrix_routing = np.zeros((num_locations, num_locations))

    for i in range(num_locations):
        for j in range(i+1, num_locations):
            distance = get_route(locations[i], locations[j])
            if isinstance(distance, (float, int)):
                distance_matrix_routing[i][j] = distance
                distance_matrix_routing[j][i] = distance

    return distance_matrix_routing.tolist()

### Functions for subtour elimination:

No need to understand this function :)

In [81]:
def SolVal(x):
  if type(x) is not list:
    if (x is None):
      #print("A")
      return 0
    elif (isinstance(x,(int,float))):
      #print("B")
      return x
    elif (x.Integer() is False):
      #print("C")
      return x.SolutionValue()
    else:
      #print(f"D: {x.SolutionValue()}")
      return int(x.SolutionValue())
  elif type(x) is list:
    #print("E")
    return [SolVal(e) for e in x]

This function gives back the different subtours within the route:

In [82]:
def extract_tours(R, n):
  #print(f"Extract_tours entered")
  #time.sleep(1)
  node = 0
  tours = [[0]]
  allnodes = [0]+[1]*(n-1)
  # We iterate until the number of tours returned by the solver is one,
  # taking care to accumulate subtours as they are discovered
  #k = 1
  while sum(allnodes) > 0:
    #print(f"\tIteration {k}")
    #k += 1
    #Another way of writing this for loop -> next = [i for i in range(n) if R[node][i]==1][0]
    for i in range(n):
      if R[node][i]==1:
        next = [i][0]
        #print(f"{next} is visited by {node}.")
        #time.sleep(0.5)
    if next not in tours[-1]:
      tours[-1].append(next)
      node = next
      #print(f"{next} is added to the tour {tours[-1]}")
      #time.sleep(0.5)
    else:
      node = allnodes.index(1)
      tours.append([node])
      #print(f"The tour is closed.")
      #print(f"A new tour is opened with {node} at the beginning.")
      #time.sleep(0.5)
    allnodes[node] = 0
    if (len(tours[-1]) == 7):
      print(f"Complete tour achieved.")
      #time.sleep(1)
  return tours

## Import Data

Import the data of the tour:

In [83]:
# import data
location_df = pd.read_csv("https://raw.githubusercontent.com/JohannG3/DSML_EPFL_Rolex/main/df_pop_fans.csv")
us_location_df = location_df

# define the data used in the model
location_names = us_location_df['city'].tolist()
locations_flight = [f"{lat},{lon}" for lat, lon in zip(us_location_df['latitude'], us_location_df['longitude'])]
locations_routing = [f"{lon},{lat}" for lon, lat in zip(us_location_df['longitude'], us_location_df['latitude'])]

In [84]:
us_location_df

,tour name,venue name,city,region,country,continent,tour dates,venue max capacity,latitude,longitude,city total population,Score,Population fan
0,The Eras Tour,State Farm Stadium,Glendale,Arizona,United States,North America,"17.03.23,18.03.23",78600,33.5791,-112.2311,248083.0,50.8,126026.164
1,The Eras Tour,Allegiant Stadium,Paradise,Nevada,United States,North America,"24.03.23,25.03.23",71835,36.0872,-115.1355,189733.0,0.0,0.000
2,The Eras Tour,AT&T Stadium,Arlington,Texas,United States,North America,"31.03.23,01.04.23,02.04.23",71835,32.6998,-97.1250,393469.0,63.5,249852.815
3,The Eras Tour,Raymond James Stadium,Tampa,Florida,United States,North America,"13.04.23,14.04.23,15.04.23",75000,27.9945,-82.4447,2861173.0,98.7,2823977.751
4,The Eras Tour,NRG Stadium,Houston,Texas,United States,North America,"21.04.23,22.04.23,23.04.23",80000,29.7860,-95.3885,5970127.0,2.9,173133.683
5,The Eras Tour,Mercedes-Benz Stadium,Atlanta,Georgia,United States,North America,"28.04.23,29.04.23,30.04.23",75000,33.7628,-84.4220,5180179.0,27.5,1424549.225
6,The Eras Tour,Nissan Stadium,Nashville,Tennessee,United States,North America,"05.05.23,06.05.23,07.05.23",69143,36.1715,-86.7842,1177657.0,57.0,671264.490
7,The Eras Tour,Lincoln Financial Field,Philadelphia,Pennsylvania,United States,North America,"12.05.23,13.05.23,14.05.23",69879,40.0077,-75.1339,5683533.0,14.7,835479.351
8,The Eras Tour,MetLife Stadium,East Rutherford,New Jersey,United States,North America,"26.05.23,27.05.23,28.05.23",82500,40.8179,-74.0854,10020.0,25.5,2555.100
9,The Eras Tour,Soldier Field,Chicago,Illinois,United States,North America,"02.06.23,03.06.23,04.06.23",62500,41.8375,-87.6866,8497759.0,14.5,1232175.055


Create the distance matrix:

In [85]:
num_locations = len(location_names)
DistanceMatrix = create_distance_matrix_routing(location_names, locations_routing)

## Optimisation of the emissions

### Optimization

**Here, you can define the parameters of the algorithm.**

This way, you won't have to look them up in the code anymore

In [94]:
# CONTROL PARAMETERS (1 means yes, 0 means no)

# For every step, do you want to show...
# which locations are opened?
showU = 0
# which location is visited after which?
showY = 0
# from which city do people travel to which?
showC = 0

# What are the maximum days of the tour?
T_total = 200

# How many hours does Taylor travel max per day?
hours_per_trip = 6

# How many emissions are associated to open one facility (= stadium)?
stadium_opening_cost = 5000000000 # g of CO2
#stadium_opening_cost = 500000000 # festival of 80,000 people for 3 days = 100tCO2

# Of the total inhabitants of the city, what percentage actually visits the concert?
attendance_rate = 0.4

# How much should people maximum travel to see a Taylor Swift concert?
maxDistance = 1000

# How many shows does Taylor perform maximum in one city?
maxDays = 4

If you want to look at Taylor's actual tour:

In [87]:
# Do you want to see the emissions of the actual ERAS tour?
actual_tour = 0

# How many people should visit the concerts in total?
people_to_reach = 4141065 #

# What are Taylor Swifts emissions before optimization?
taylors_emissions = 117785653154

Parameter initialization:

In [88]:
import time
from ortools.linear_solver import pywraplp

transport_types = ['Bus (average)', 'Coach (bus)', 'Diesel car', 'Electric car',
                   'Motorbike', 'National rail', 'Petrol car', 'Plug-in hybrid', 'Short-haul flight', 'Medium-haul flight',
                   'Long-haul flight', 'Private Jet (Short-haul) flight', 'Private Jet (Medium-haul) flight', 'Private Jet (Long-haul) flight']
long_flights = ['Long-haul flight','Private Jet (Long-haul) flight']
medium_flights = ['Medium-haul flight','Private Jet (Medium-haul) flight']
short_flights = ['Short-haul flight','Private Jet (Short-haul) flight']
private_jets = ['Private Jet (Long-haul) flight','Private Jet (Medium-haul) flight','Private Jet (Short-haul) flight']
num_types = len(transport_types)

# Create a dummy for size of cities and size of stadiums
capacity = us_location_df["venue max capacity"].tolist()
citizens = us_location_df["Population fan"].tolist()

In [89]:
def solve_model(DistanceMatrix):
    subtours, tours = [], []
    # Evaluate whether there are subtours:
    while len(tours) != 1:
        status, Value, tours, storage, y_storage, time_storage, stadium_storage, city_storage, show_storage = solve_model_eliminate(DistanceMatrix, subtours)
        if status == pywraplp.Solver.OPTIMAL:
            subtours.extend(tours)
        print("Tours:", tours)
    return status, Value, tours[0], storage, y_storage, time_storage, stadium_storage, city_storage, show_storage

In [98]:
def solve_model_eliminate(DistanceMatrix, Subtours = []):
    """

    Args:
      DistanceMatrix:
      Subtours:

    Returns:

    """
    solver = pywraplp.Solver.CreateSolver('CBC')
    n = len(DistanceMatrix)
    num_types = len(transport_types)

    # VARIABLE DECLARATION

    # From i to j, Taylor takes vehicle k
    x = [[[solver.IntVar(0, 1, '') for k in range(num_types)] for j in range(n)] for i in range(n)]   # NEED TO MODIFY THE CONSTRAINT OF VEHICULE TYPE FOR TAYLOR
    # Taylor goes from i to j
    y = [[solver.IntVar(0, 1, '') for j in range(n)] for i in range(n)]
    # Taylor takes this many days
    t = [[solver.IntVar(0, 10000, '') for j in range(n)] for i in range(n)]
    # Stadium in city j is opened
    u = [solver.IntVar(0, 1, '') for j in range(n)]
    # City i travels to stadium j
    #d = [[solver.IntVar(0,10000,'') for j in range(n)] for i in range(n)]
    c = [[None for j in range(n)] for i in range(n)]
    for i in range(n):
      for j in range(n):
        if location_names[i] == location_names[j]:
            c[i][j] = solver.IntVar(0, citizens[i], f'c_{i}_{j}')
        else:
            c[i][j] = solver.IntVar(0, 20000, f'c_{i}_{j}')

    # Taylor performs s shows in city j
    s = [solver.IntVar(0, maxDays, f's_{j}') for j in range(n)]

    # COMPARE WITH THE ACTUAL TOUR
    tour_names_2023 = ["Glendale", "Paradise", "Arlington", "Tampa", "Houston", "Atlanta", "Nashville", "Philadelphia", "Foxborough", "East Rutherford", "Chicago", "Detroit", "Pittsburgh", "Minneapolis", "Cincinnati", "Kansas City", "Denver", "Seattle", "Santa Clara", "Los Angeles"]
    if actual_tour == 1:
      # taylor cannot travel between cities that are not part of the tour
      tour_indices = []
      for i in range(len(tour_names_2023)):
        tour_indices.append(location_names.index(tour_names_2023[i]))
      for i in range(n):
        for j in range(n):
          if not(i in tour_indices):
            if not(j in tour_indices):
              for k in range(num_types):
                solver.Add(x[i][j][k] == 0)
          # also, she has to keep the right order of the venues:
          else:
            tour_index = tour_names_2023.index(location_names[i])
            if tour_index < len(tour_names_2023) - 1:
              next_index = location_names.index(tour_names_2023[tour_index+1])
              if not(j == next_index):
                for k in range(num_types):
                  solver.Add(x[i][j][k] == 0)
      # Only cities that are part of the tour are open:
      for index, name in enumerate(location_names):
        is_in_tour = 0
        for tour_name in tour_names_2023:
          if name == tour_name and is_in_tour == 0:
            is_in_tour = 1
            solver.Add(u[index] == 1)
      # Only private jet for Taylor:
      for j in range(n):
        for i in range(n):
          for k in range(num_types):
            is_private_jet = 0
            for private_jet in private_jets:
              if transport_types[k] == private_jet:
                is_private_jet = 1
            if is_private_jet == 0:
              solver.Add(x[i][j][k] == 0)

    # CONSTRAINTS

    # Differentiate between short, medium and long flights
    for i in range(n):
        for j in range(n):
          for k in range(num_types):
            if DistanceMatrix[i][j] >= 4000:
              if any(item==transport_types[k] for item in short_flights) or any(item==transport_types[k] for item in medium_flights):
                solver.Add(x[i][j][k] == 0)
            elif DistanceMatrix[i][j] <= 1500:
              if any(item==transport_types[k] for item in medium_flights) or any(item==transport_types[k] for item in long_flights):
                solver.Add(x[i][j][k] == 0)
            else:
              if any(item==transport_types[k] for item in short_flights) or any(item==transport_types[k] for item in long_flights):
                solver.Add(x[i][j][k] == 0)

    # Couple y with x
    for i in range(n):
        for j in range(n):
            solver.Add(y[i][j] - sum(x[i][j][k] for k in range(num_types)) == 0)

    # Couple t with x
    for i in range(n):
      for j in range(n):
          solver.Add(t[i][j] - sum(x[i][j][k]*round(DistanceMatrix[i][j]/hours_per_trip/speed(transport_types[k],DistanceMatrix[i][j])) for k in range(num_types)) == 0)

    # One node is only entered and exited once, if it the node (i.e. the stadium) is actually used
    for i in range(n):
        solver.Add(sum(y[i][j] for j in range(n)) == u[i])
        solver.Add(sum(y[j][i] for j in range(n)) == u[i])
        solver.Add(y[i][i] == 0)

    # time constraints:
    solver.Add(solver.Sum(t[i][j] for j in range(n) for i in range(n)) + solver.Sum(s[j] for j in range(n)) <= T_total)

    # subtour constraints
    for sub in Subtours:
        K = [y[sub[i]][sub[j]] + y[sub[j]][sub[i]] for i in range(len(sub)-1) for j in range(i+1,len(sub))]
        solver.Add(sum(K) <= len(sub)-1)

    # One city cannot send more people to concerts than it has fans
    for i in range(n):
      solver.Add(sum(c[i][j] for j in range(n)) <= citizens[i]*attendance_rate)

    # A stadium cannot accept more people than its capacity times the number of shows
    for j in range(n):
        solver.Add(solver.Sum(c[i][j] for i in range(n)) <= s[j] * capacity[j])

    # There can only be concerts in an open venue
    for j in range(n):
      solver.Add(u[j] <= s[j])
      solver.Add(s[j] <= maxDays*u[j])

    # A city only sends people to a stadium below a certain distance
    for i in range(n):
      for j in range(n):
        if DistanceMatrix[i][j] >= maxDistance and not(i == j):
          solver.Add(c[i][j] == 0)

    # The whole tour has to reach a certain number of people:
    if not actual_tour == 1:
      solver.Add(sum(c[i][j] for j in range(n) for i in range(n)) >= people_to_reach) #current_attendance)

    # objective function to minimize emissions
    objective = solver.Objective()

    if actual_tour == 1:
      for j in range(n):
        for i in range(n):
          objective.SetCoefficient(c[i][j], -1)  # Maximize attendance
    else:
      # Minimize the ecological impact of...
      # Taylor's travel
      for k in range(num_types):
        for j in range(n):
          for i in range(n):
            objective.SetCoefficient(x[i][j][k], ecological_impact(DistanceMatrix[i][j], transport_types[k]))

      # people travelling
      for j in range(n):
        for i in range(n):
            objective.SetCoefficient(c[i][j], ecological_impact(DistanceMatrix[i][j], transport_types[1]))

      # opening a stadium
      for j in range(n):
        objective.SetCoefficient(u[j], stadium_opening_cost)


    # and solve
    status = solver.Solve()

    # store the values of the decision variables
    storage = []
    time_storage = []
    stadium_storage = []
    city_storage = []
    y_storage = []
    show_storage = []

    for i in range(n):
        storageI = []
        time_storageI = []
        city_storageI = []
        y_storageI = []
        if showU == 1:
          print(f"u[{i}] = ", u[i].solution_value())
          print(f"s[{i}] (shows) = ", s[i].solution_value())
        for j in range(n):
            storageJ = []
            if showY == 1:
              print(f"y[{i}][{j}] = ",y[i][j].solution_value())
            if showC == 1:
              print(f"d[{j}][{i}] = ",c[j][i].solution_value())
            for k in range(num_types):
                storageJ.append(x[i][j][k].solution_value())
            storageI.append(storageJ)
            time_storageI.append(t[i][j].solution_value())
            city_storageI.append(c[i][j].solution_value())
            y_storageI.append(y[i][j].solution_value())
        storage.append(storageI)
        time_storage.append(time_storageI)
        stadium_storage.append(u[i].solution_value())
        city_storage.append(city_storageI)
        show_storage.append(s[i].solution_value())


    solval = [[y[i][j].solution_value() for j in range(n)] for i in range(n)]
    #print(solval)
    tours = extract_tours(solval, n, stadium_storage)
    return status, solver.Objective().Value(), tours, storage, y_storage, time_storage, stadium_storage, city_storage, show_storage

In [91]:
def extract_tours(solval, n, stadium_storage):
    tours = []
    visited = [False] * n
    for i in range(n):
        if (not visited[i]) and (stadium_storage[i] == 1):
            tour = []
            current = i
            while not visited[current]:
                tour.append(current)
                visited[current] = True
                for j in range(n):
                    if solval[current][j] == 1:
                        current = j
                        break
            tours.append(tour)
    return tours

In [97]:
# define model: you can use CpModel()
start = time.time()

status, value, tour, storage, y_storage, time_storage, stadium_storage, city_storage, show_storage = solve_model(DistanceMatrix)
if status == pywraplp.Solver.OPTIMAL:
    print('Minimum impact =', round(value))
    taylor_transport_emissions = 0
    stadium_emissions = 0
    people_travel_emissions = 0
    total_days = 0

    # Calculations regarding Taylor's tour
    for i in tour:
        for j in range(num_locations):
            if i != j and any(storage[i][j]):
                for k in range(num_types):
                    if storage[i][j][k] == 1:
                        distance = DistanceMatrix[i][j]
                        transport = transport_types[k]
                        # Emission calculation
                        emission = ecological_impact(distance, transport)
                        taylor_transport_emissions += emission
                        # Time calculation
                        days = time_storage[i][j]
                        total_days += days
                        # Increase the total time at the location by the number of shows minus one (since the initial stay is already counted)
                        extra_days_due_to_shows = show_storage[i]
                        total_days += extra_days_due_to_shows
                        print(f"Taylor travels {round(distance)} km from {location_names[i]} to {location_names[j]} by {transport}. Emissions: {round(emission)} grams CO2. Time {round(days)} days.")
        # If there are shows in the current city of the tour, consider those as well
        if show_storage[i] >= 1:  # Use show_storage here
            extra_days_at_start = show_storage[i] - 1
            total_days += extra_days_at_start
            print(f"Starting at {location_names[i]} with {show_storage[i]} shows, adding {extra_days_at_start} extra days.")
    print(f"Total time of the tour: {round(total_days)} days.")
    print("\n")

    # Calculations concerning opening a stadium and people's travel to it
    people_reached = 0
    for i in tour:
        print(f"Taylor performs {show_storage[i]} shows in {location_names[i]} with {capacity[i]} seats for a total of {show_storage[i]*capacity[i]} seats. It is visited by:")
        people_in_stadium = 0
        # From which cities come people to the stadium in question? How much do they emit?
        for j in range(num_locations):
          if city_storage[j][i] > 0:
            people_emissions = city_storage[j][i] * ecological_impact(DistanceMatrix[i][j],transport_types[1])
            print(f"\t{round(city_storage[j][i])} people from {location_names[j]} {'by ' + transport_types[1] if i != j else ''} -> {round(people_emissions)}g of CO2 emitted")
            people_in_stadium += city_storage[j][i]
            people_travel_emissions += people_emissions
        print(f"\tAttendance in {location_names[i]}: {round(people_in_stadium)} people (attendance rate of {round(people_in_stadium/(show_storage[i]*capacity[i])*100)}%).\n")
        # How many people are reached by this stadium?
        people_reached += people_in_stadium
        # Add the cost of opening one stadium
        stadium_emissions += stadium_opening_cost
    total_emissions = stadium_emissions + taylor_transport_emissions + people_travel_emissions
    # Display the results
    print(f"\nTotal attendance: {round(people_reached)} people.")
    print(f"Emissions from Taylors travel: {round(taylor_transport_emissions)} grams CO2.")
    print(f"Emissions from peoples travel: {round(people_travel_emissions)} grams CO2.")
    print(f"Emissions from stadiums: {round(stadium_emissions)} grams CO2.")
    print(f"Total emissions for the tour: {round(total_emissions)} grams CO2.")
    #print(f"FINAL RESULT: A reduction of {round((1-total_emissions/taylors_emissions)*100)} %.")     # DOES NOT REALLY MAKE SENSE TO COMPARE TAYLOR TO GLOBAL
    print(f"FINAL RESULT: The emissions for Taylor transport go from {round(taylors_emissions)} to {round(taylor_transport_emissions)} grams CO2, thus a reduction of {round((1-taylor_transport_emissions/taylors_emissions)*100, 4)} %.")

    if actual_tour == 1:
      taylors_emissions = total_emissions
      people_to_reach = people_reached
else:
    print('The problem does not have an optimal solution.')
end = time.time()
print("Time of computation =", round(end - start, 4), "seconds")

Tours: [[0, 15], [2, 4], [3, 16], [5, 6], [7, 8, 19], [9, 21], [10, 20], [11, 18], [12, 13], [14, 22]]
Tours: [[0, 14, 22, 15], [2, 4, 13], [3, 5, 16], [6, 11, 18], [7, 20], [8, 19], [9, 10], [12, 21]]
Tours: [[0, 15, 22, 14, 13], [2, 12, 21, 9, 10, 11, 18, 6, 5, 16, 3, 4], [7, 8, 19, 20]]
Tours: [[0, 2, 4, 3, 16, 5, 6, 11, 18, 9, 21, 12, 13, 14, 22, 15], [7, 8, 19, 10, 20]]
Tours: [[0, 2, 4, 3, 16, 5, 6, 18, 11, 20, 7, 8, 19, 10, 9, 21, 12, 13, 14, 22, 15]]
Minimum impact = 114712771150
Taylor travels 1709 km from Glendale to Arlington by Coach (bus). Emissions: 46708 grams CO2. Time 4 days.
Starting at Glendale with 1.0 shows, adding 0.0 extra days.
Taylor travels 408 km from Arlington to Houston by Coach (bus). Emissions: 11137 grams CO2. Time 1 days.
Starting at Arlington with 2.0 shows, adding 1.0 extra days.
Taylor travels 1577 km from Houston to Tampa by Coach (bus). Emissions: 43112 grams CO2. Time 4 days.
Starting at Houston with 1.0 shows, adding 0.0 extra days.
Taylor travel

In [ ]:
import folium

complete_df = pd.read_csv("https://raw.githubusercontent.com/JohannG3/DSML_EPFL_Rolex/main/df_pop_fan_map.csv")
tour_names = [location_names[i] for i in tour]
tour_string = " -> ".join(tour_names)
print("Tour:", tour_string)

# Map centered on US
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add a Choropleth layer for U.S. data
folium.Choropleth(
    geo_data='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json',
    name='choropleth',
    data=complete_df,
    columns=['State', 'Population fan'],
    key_on='feature.properties.name',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    color='pink',
    legend_name='Number of people fan of Taylor Swift per State'
).add_to(m)

# GeoJSON for Canada
canada_geojson_url = 'https://raw.githubusercontent.com/JohannG3/DSML_EPFL_Rolex/main/georef-canada-province%40public.geojson?raw=true'

# GeoJSON for Mexico
mexico_geojson_url = 'https://raw.githubusercontent.com/JohannG3/DSML_EPFL_Rolex/main/georef-mexico-state-millesime%40public.geojson?raw=true'

# Add overlay layers for Canada
folium.GeoJson(
    data=canada_geojson_url,
    style_function=lambda x: {'fillColor': 'white', 'color': 'black', 'fillOpacity': 1, 'weight': 0},
    name='Canada'
).add_to(m)

# Add overlay layers for Mexico
folium.GeoJson(
    data=mexico_geojson_url,
    style_function=lambda x: {'fillColor': 'white', 'color': 'white', 'fillOpacity': 1, 'weight': 0},
    name='Mexico'
).add_to(m)

# Add a layer control panel to toggle layers
folium.LayerControl().add_to(m)

# Tracing the tour and emissions on the map
for i in tour:
    stadium_name = location_names[i]
    people_in_stadium = 0
    people_travel_emissions = 0
    for j in range(num_locations):
        if city_storage[j][i] > 0:
            # Draw lines on the map
            start_city = location_names[j]
            end_city = stadium_name
            start_loc = location_df[location_df['city'] == start_city].iloc[0]
            end_loc = location_df[location_df['city'] == end_city].iloc[0]
            start_lat, start_lon = start_loc['latitude'], start_loc['longitude']
            end_lat, end_lon = end_loc['latitude'], end_loc['longitude']
            folium.PolyLine(locations=[(start_lat, start_lon), (end_lat, end_lon)], color='red').add_to(m)

# Tracing the tour on the map
for i in range(len(tour_names) - 1):
    start_city = tour_names[i]
    end_city = tour_names[i + 1]

    start_loc = location_df[location_df['city'] == start_city].iloc[0]
    end_loc = location_df[location_df['city'] == end_city].iloc[0]

    start_lat, start_lon = start_loc['latitude'], start_loc['longitude']
    end_lat, end_lon = end_loc['latitude'], end_loc['longitude']

    folium.Marker(location=[start_lat, start_lon], popup=start_city).add_to(m)
    folium.Marker(location=[end_lat, end_lon], popup=end_city).add_to(m)
    folium.PolyLine(locations=[(start_lat, start_lon), (end_lat, end_lon)], color='blue').add_to(m)

# Map (download and open it on the web)
m.save('tour_map_us_opt_FLP_BUS.html')

**To view the visualization, simply download the .html file and click it.**